In [1]:
import numpy as np
from cs231n.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

**Sample Data**

In [16]:
# Stride and Pad
(S, P) = (2, 1)
# Input Channels, HeightIn, WidthIn
(C, HI, WI) = (3, 5, 5)
# Filters, HeightFilter, WidthFilter
(F, HF, WF) = (2, 3, 3)
# Padded image dimensions
(HP, WP) = HI + 2 * P, WI + 2 * P
print "HP, WP", HP, WP
# Output image HeightOut, WidthOut
(HO, WO) = ((HP - HF)/S + 1, (WP - WF)/S + 1) 
print "HO, WO", HO, WO

x= np.empty((C, HI, WI))
x[0, :, :] = np.array([
    [2, 0, 2, 2, 0], 
    [1, 2, 2, 2, 2], 
    [0, 0, 1, 0, 0], 
    [2, 1, 0, 2, 1], 
    [0, 1, 0, 1, 2], 
])

x[1, :, :] = np.array([
    [2, 0, 1, 2, 0], 
    [2, 0, 2, 0, 1], 
    [2, 1, 1, 2, 1], 
    [2, 1, 1, 2, 2], 
    [2, 0, 0, 0, 2], 
])

x[2, :, :]  = np.array([
    [2, 2, 2, 2, 0], 
    [1, 0, 0, 1, 2], 
    [2, 1, 2, 2, 1], 
    [2, 0, 2, 2, 1], 
    [1, 0, 0, 2, 1], 
])

# Filters, Channels, HF, WF
w = np.empty((2, 3, 3, 3))
w[0, 0, :, :] = np.array([
        [0, 0, -1],
        [0, 1, 0],
        [1, 0, 0]
    ])
w[0, 1, :, :] = np.array([
        [1, 1, 1],
        [-1, 1, 1],
        [1, 0, -1]
    ])
w[0, 2, :, :] = np.array([
        [0, 1, -1],
        [0, 1, 1],
        [0, -1, 1]
    ])

w[1, 0, :, :] = np.array([
        [-1, 0, 1],
        [0, 1, -1],
        [0, 1, 1]
    ])
w[1, 1, :, :] = np.array([
        [1, -1, 0],
        [1, 0, 1],
        [0, 1, -1]
    ])
w[1, 2, :, :] = np.array([
        [-1, -1, 1],
        [0, -1, -1],
        [-1, 0, 0]
    ])
b = np.array([1, 0])

correct_out = np.array([
        [[8, 13, -1],
        [5, 7, 7],
        [8, 5, 11]],
        [[3, 4, 4],
        [1, 0, -4],
        [-5, -2, -4]],
    ])

HP, WP 7 7
HO, WO 3 3


** Forward pass **

In [34]:
x_pad = np.pad(x, ((0, 0), (P, P), (P, P)), 'constant')
out = np.empty((F, HO, WO))
for f in range (F):
    for i in range(HO):
      for j in range(WO):
            out[f, i, j] = np.sum(x_pad[:, i*S:i*S+HF, j*S:j*S+WF] * w[f]) + b[f]

print 'Forward pass error'
print 'error: ', rel_error(out, correct_out)
# print out
# print correct_out

Forward pass error
error:  0.0


** Forward pass in steps **

In [18]:
def pad_x(x, P):
    """
    Pad x with P zeros before and after image dimensions
    Input 
        - x (C, H, W)
    Output
        - x(C, H + 2P, W + 2P)
    """
    return np.pad(x, ((0, 0), (P, P), (P, P)), 'constant')


def distribute_x(x, HF, WF, S):
    '''
    Distribute input image to convolution ready sub-images.  
    Make a filter ready image for each filter and output image dimension 
    Inputs:
    - x (C, H, W) input image
    Output
    - out (F, HO, WO, C, HF, WF)
    -- F number of filters
    -- HO, WO, output dimensions
    -- C Input channels
    -- HF WF filter dimensions
    '''
    (C, HI, WI) = x.shape
    HO = 1 + (HI - HF) / S
    WO = 1 + (WI - WF) / S
    out = np.empty((F, HO, WO, C, HF, WF))

    for f in range(F):
        for i in range(HO):
            for j in range(WO):
                out[f, i, j] = x[:, i*S:i*S+HF, j*S:j*S+WF]
    return out

def multiply_w(x, w):
    '''
    Apply filters to (C, HF, WF) sections
    Inputs
    - x (F, HO, WO, C, HF, WF)
    - w (F, C, HF, WF)
    Returns
    - out (F, HO, WO, C, HF, WF)
    '''
    out = np.empty((F, HO, WO, C, HF, WF))
    for f in range(F):
        for i in range(HO):
            for j in range(WO):
                out[f, i, j] = x[f, i, j] * w[f]
    return out
    
def sum_filter(x):
    '''
    For each out pixel sum up weighted x*w over (C, HF, WF)
    Input
    - x (F, HO, WO, C, HF, WF)
    Output
    - out (F, HO, WO)
    '''
    out = np.empty((F, HO, WO))
    for f in range(F):
        for i in range(HO):
            for j in range(WO):
                out[f, i, j] = np.sum(x[f, i, j])
    return out

def add_bias(x, b):
    '''
    Apply the bias terms
    Inputs
    - x (F, HO, WO)
    - b (F,)
    '''
    out = np.empty_like(x)
    for f in range(F):
        out[f] = x[f] + b[f]
    return out


### Test back prop on pad

In [24]:
dout = np.random.randn(C, HP, WP)

# evaluate numerical gradient
dx_num = eval_numerical_gradient_array(lambda x: pad_x(x, P), x, dout)
# print "dx_num\n", dx_num

# evalute gradient via backpropagation - basically trim off padding
dx = dout[:, 1:HI + 2 * P - 1, 1:WI + 2 * P - 1]
# print "dx\n", dx

print 'Testing pad_x function'
print 'dx error: ', rel_error(dx, dx_num)


Testing pad function
dx error:  3.27562492402e-12


### test backprop on distribute

In [40]:

x_pad = pad_x(x, P)   
x_dist = distribute_x(x_pad, HF, WF, S)
dout = np.random.randn(F, HO, WO, C, HF, WF)

print "x_dist.shape", x_dist.shape
print "dout.shape", dout.shape

# evaluate numerical gradient
dx_num = eval_numerical_gradient_array(lambda x: distribute_x(x, HF, WF, S), x_pad, dout)
print "x_pad.shape", x_pad.shape
print "dx_num.shape", dx_num.shape


#x_dist (F, HO, WO, C, HF, WF)
# for each (F, HO, WO)
# identify x(i, j) in filter
# map from x(i, j) to (C, HF, WF)
# dx(i, j) += dxd(F, HO, WO, C, HF, WF)
dx = np.zeros_like(x_pad)
print "dx.shape", dx.shape

# for each output (F, HO, WO)
for f in range(F):
    for y_out in range(HO):
        for x_out in range (WO):
            # get the upper left filter input coordinates
            (y_in_ul, x_in_ul) = (y_out * S, x_out * S)
            for y_filter in range(HF):
                for x_filter in range(WF):
                    (y_in, x_in) = (y_in_ul + y_filter, x_in_ul + x_filter)
                    #for c in range(C):
                    dx[:, y_in, x_in] += dout[f, y_out, x_out, :, y_filter, x_filter]
 
print 'Testing distribute_x function'
print 'dx error: ', rel_error(dx, dx_num)
# print "dx\n", dx
# print "dx_num\n", dx_num

x_dist.shape (2, 3, 3, 3, 3, 3)
dout.shape (2, 3, 3, 3, 3, 3)
x_pad.shape (3, 7, 7)
dx_num.shape (3, 7, 7)
dx.shape (3, 7, 7)
Testing distribute_x function
dx error:  3.27569699424e-12
dx
[[[  9.94999833e-01  -1.45621154e+00   4.94823828e-01  -3.34253366e-01
    -2.97809128e+00   1.34647043e+00   7.54019188e-01]
  [ -2.69812219e+00  -1.06716742e+00  -1.63504364e+00  -1.52108634e+00
     5.93771292e+00   1.63802732e+00  -7.07678722e-01]
  [ -2.17451261e+00  -2.66136357e+00  -6.04893339e+00  -2.16221966e+00
     2.71270815e+00   1.66635205e-01   1.58415605e+00]
  [ -7.67222645e-01  -1.77252094e+00  -5.56662271e-01  -1.03628214e+00
    -2.10640878e+00   1.02211310e+00   3.36137732e+00]
  [  2.18638303e+00   1.16140405e+00   3.53030883e-01  -1.57707916e+00
     1.13832034e+00   4.94002528e-01   2.61656434e+00]
  [  2.67142320e+00   1.49487600e-01   1.60951468e+00   2.57589039e-01
    -7.56653465e-01   4.26185630e-02  -1.30286987e+00]
  [  2.93046633e+00  -1.89515832e+00   1.18429021e+00  -